# Processing Data for Modeling

## Notebook Intro

This is the second notebook in a total of 3.

In this notebook we will be:
1. Processing all Categorical Columns into Numeric
2. Removing Highly Correlated Columns 

Get most important features and get rid of highly correlated columns

### General Imports

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Load Data

Let's load in our previously cleaned data:

In [2]:
# read in non-null data
accepted_nonull = pd.read_csv('accepted_nonull.csv')

## Quick Sanity Checks

Before we start processing our columns, lets check the shape of the data and make sure that we still have 0 NULL values.

In [3]:
accepted_nonull.shape

(1972940, 68)

In [4]:
accepted_nonull.isna().sum().sum()

0

Our file looks good and we can move onto processing. 

## Processing the Categorical Columns

First, we are going to be splitting our data into Categorical and Numerical data frames.

In [5]:
categorical_df = accepted_nonull.select_dtypes('object').copy()
numerical_df = accepted_nonull.select_dtypes('number').copy()

In [6]:
print(categorical_df.shape)
print(numerical_df.shape)

(1972940, 11)
(1972940, 57)


After the split, we have 11 categorical columns and 57 numerical columns. Let's take a look at the first 5 rows of each.

In [7]:
numerical_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,earliest_cr_years_ago
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,12.0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,16.0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,15.0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,7.0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,17.0


In [8]:
categorical_df.head()

,term,grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,initial_list_status,application_type,disbursement_method
0,36 months,C,10+ years,MORTGAGE,Not Verified,Paid,debt_consolidation,PA,w,Individual,Cash
1,36 months,C,10+ years,MORTGAGE,Not Verified,Paid,small_business,SD,w,Individual,Cash
2,60 months,B,10+ years,MORTGAGE,Not Verified,Paid,home_improvement,IL,w,Joint App,Cash
3,60 months,C,10+ years,MORTGAGE,Source Verified,Paid,debt_consolidation,NJ,w,Individual,Cash
4,60 months,F,3 years,MORTGAGE,Source Verified,Paid,major_purchase,PA,w,Individual,Cash


### Term

*Data Dictionary*
- term: The number of payments on the loan. Values are in months and can be either 36 or 60.

The first column we are going to look at is `term`. At first glance, this columns looks to be numeric and we just have to remove the word "months". Let's take a look at the values in this column.

In [9]:
categorical_df['term'].value_counts()

 36 months    1391794
 60 months     581146
Name: term, dtype: int64

We only have two values, 36 and 60 months. We are going to split the column by the white space, and then extract the number. Once we extract the number we can convert the column to an integer.

In [10]:
# split term column by space
categorical_df['term'] = categorical_df['term'].str.split(' ').str[1]

# convert term column to int
categorical_df['term'] = categorical_df['term'].astype(int)

# get new value counts
categorical_df['term'].value_counts()

36    1391794
60     581146
Name: term, dtype: int64

Now that the column is processed, we can drop it from our categorical data frame and add it to our numerical data frame.

In [11]:
# drop term from categorical_df and add to numerical_df
numerical_df['term'] = categorical_df['term']
categorical_df = categorical_df.drop(columns='term')

### Grade

The next column we are going to look at is `grade`.

*Data Dictionary*
- grade: LC assigned loan grade

Let's take a look at the values in this feature:

In [12]:
categorical_df['grade'].value_counts()

B    581431
C    570566
A    380318
D    278494
E    116184
F     35718
G     10229
Name: grade, dtype: int64

As mentioned in the previous notebook, the grade of a loan determines the expected risk of loss, with "A" being the least risky to "G" being the most risky. We can ordinal encode `grade` to show that "G" is more valuable than "A".

In [13]:
# import ordinal encoder
from sklearn.preprocessing import OrdinalEncoder

# ordinal encode grade column
ordinal_encoder = OrdinalEncoder()
categorical_df['grade'] = ordinal_encoder.fit_transform(categorical_df[['grade']])

# get new value counts
categorical_df['grade'].value_counts()


1.0    581431
2.0    570566
0.0    380318
3.0    278494
4.0    116184
5.0     35718
6.0     10229
Name: grade, dtype: int64

Now that the column is processed, we can drop it from our categorical data frame and add it to our numerical data frame.

In [14]:
# drop grade from categorical_df and add to numerical_df
numerical_df['grade'] = categorical_df['grade']
categorical_df = categorical_df.drop(columns='grade')

### Emp Length

Next, we can look at `emp_length`.

*Data Dictionary*
- emp_length: number of years employed

Let's look at the values in this feature:

In [15]:
categorical_df['emp_length'].value_counts()

10+ years    708618
2 years      190687
3 years      169081
< 1 year     165896
1 year       138711
5 years      129444
4 years      126686
6 years       95388
7 years       86744
8 years       86644
9 years       75041
Name: emp_length, dtype: int64

To convert this column into a numerical value, we are going to map the current values to numbers and convert to numeric. 

In [16]:
# emp_length mapper
emp_length_mapper = {'< 1 year': 0, '1 year': 1, 
                     '2 years': 2, '3 years': 3, 
                     '4 years': 4, '5 years': 5, 
                     '6 years': 6, '7 years': 7, 
                     '8 years': 8, '9 years': 9, 
                     '10+ years': 10}

# apply mapper to emp_length column
categorical_df['emp_length'] = categorical_df['emp_length'].map(emp_length_mapper)

# value counts 
categorical_df['emp_length'].value_counts()

10    708618
2     190687
3     169081
0     165896
1     138711
5     129444
4     126686
6      95388
7      86744
8      86644
9      75041
Name: emp_length, dtype: int64

In [17]:
# convert emp_length to int
categorical_df['emp_length'] = categorical_df['emp_length'].astype(int)

Now that the column is processed, we can drop it from our categorical data frame and add it to our numerical data frame.

In [18]:
# drop emp_length from categorical_df and add to numerical_df
numerical_df['emp_length'] = categorical_df['emp_length']
categorical_df = categorical_df.drop(columns='emp_length')

Let's see what we have left:

In [19]:
categorical_df.head()

,home_ownership,verification_status,loan_status,purpose,addr_state,initial_list_status,application_type,disbursement_method
0,MORTGAGE,Not Verified,Paid,debt_consolidation,PA,w,Individual,Cash
1,MORTGAGE,Not Verified,Paid,small_business,SD,w,Individual,Cash
2,MORTGAGE,Not Verified,Paid,home_improvement,IL,w,Joint App,Cash
3,MORTGAGE,Source Verified,Paid,debt_consolidation,NJ,w,Individual,Cash
4,MORTGAGE,Source Verified,Paid,major_purchase,PA,w,Individual,Cash


### Home Ownsership

The next column to process is `home_ownership`.

*Data Dictionary*
- home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report.

Let's look at the values in this feature:

In [20]:
categorical_df['home_ownership'].value_counts()

MORTGAGE    980382
RENT        783505
OWN         208262
ANY            709
NONE            42
OTHER           40
Name: home_ownership, dtype: int64

Looking at the unique values in this feature, we can see that "ANY", "OTHER", and "NONE" have very few values in them. We can combine all three of these into "OTHER".

In [21]:
# mapper
home_ownership_mapper = {'MORTGAGE': 'MORTGAGE', 'RENT': 'RENT', 
                         'OWN': 'OWN', 'ANY': 'OTHER', 'NONE': 
                         'OTHER', 'OTHER': 'OTHER', False: None}

# replace values
categorical_df['home_ownership'] = categorical_df['home_ownership'].replace(home_ownership_mapper)

Now that we have applied mapping to the column let's look at the new value counts:

In [22]:
# value counts for home_ownership
categorical_df['home_ownership'].value_counts()

MORTGAGE    980382
RENT        783505
OWN         208262
OTHER          791
Name: home_ownership, dtype: int64

To convert these into numeric, we will be using One Hot Encoder. This encoder will give each value its own column in the data. Each column will be binary, showing a 1 if the value applies to that loan and a 0 if it does not.

In [23]:
# import one hot encoder
from sklearn.preprocessing import OneHotEncoder

# instantiate one hot encoder
ohe = OneHotEncoder()

# one hot encode home_ownership column
home_ownership_ohe = ohe.fit_transform(categorical_df[['home_ownership']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(home_ownership_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)


Now that we have encoded the colum, we can drop `home_ownership` from our categorical data and add the 4 new columns to our numerical data.

In [24]:
# drop home_ownership column from categorical_df
categorical_df = categorical_df.drop('home_ownership', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(home_ownership_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Sanity check on numerical data:

In [25]:
numerical_df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,total_bc_limit,total_il_high_credit_limit,earliest_cr_years_ago,term,grade,emp_length,MORTGAGE,OTHER,OWN,RENT
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,2400.0,13734.0,12.0,36,2.0,10,1.0,0.0,0.0,0.0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,79300.0,24667.0,16.0,36,2.0,10,1.0,0.0,0.0,0.0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,6200.0,14877.0,15.0,60,1.0,10,1.0,0.0,0.0,0.0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,62500.0,18000.0,7.0,60,2.0,10,1.0,0.0,0.0,0.0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,20300.0,88097.0,17.0,60,5.0,3,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972935,24000.0,10.49,515.74,125000.0,10.98,0.0,725.0,729.0,0.0,15.0,...,93700.0,28602.0,15.0,60,1.0,4,0.0,0.0,1.0,0.0
1972936,24000.0,10.49,515.74,108000.0,34.94,0.0,695.0,699.0,1.0,24.0,...,27900.0,172283.0,25.0,60,1.0,10,1.0,0.0,0.0,0.0
1972937,40000.0,10.49,859.56,227000.0,12.75,7.0,705.0,709.0,1.0,5.0,...,12300.0,42670.0,21.0,60,1.0,9,1.0,0.0,0.0,0.0
1972938,24000.0,14.49,564.56,110000.0,18.30,0.0,660.0,664.0,0.0,10.0,...,20700.0,58764.0,17.0,60,2.0,6,0.0,0.0,0.0,1.0


Our numerical data frame looks good. We can now drop the encoded columns from our categorical data frame as well as drop "OTHER" from our numerical data frame. This is because when you encode a column, you can predict one of them using all the others. In this case we will be dropping "OTHER" as it had the least amount of data.

In [26]:
# drop MORTGAGE, OTHER, OWN, RENT columns from categorical_df
categorical_df = categorical_df.drop(['MORTGAGE', 'OTHER', 'OWN', 'RENT'], axis=1)

# drop OTHER column from numerical_df
numerical_df = numerical_df.drop('OTHER', axis=1)

### Verification Status

Now we can move on to processing `verification_status`.

*Data Dictionary*
- verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified

Let's look at the values in this feature:

In [27]:
categorical_df['verification_status'].value_counts()

Source Verified    811327
Not Verified       649297
Verified           512316
Name: verification_status, dtype: int64

We can see that we have 3 unique values. Although two of the columns seem to have the same outcome of "verified", source verified means that the actual source of the income was verified. This could be a valuable predictor in our model so we are going to One Hot Encode this column.

In [28]:
# one hot encode verification_status column
verification_status_ohe = ohe.fit_transform(categorical_df[['verification_status']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(verification_status_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)

Now that we have encoded the colum, we can drop `verification_status` from our categorical data and add the 3 new columns to our numerical data.

In [29]:
# drop verification_status column from categorical_df
categorical_df = categorical_df.drop('verification_status', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(verification_status_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Sanity check on numerical data:

In [30]:
numerical_df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,earliest_cr_years_ago,term,grade,emp_length,MORTGAGE,OWN,RENT,Not Verified,Source Verified,Verified
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,12.0,36,2.0,10,1.0,0.0,0.0,1.0,0.0,0.0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,16.0,36,2.0,10,1.0,0.0,0.0,1.0,0.0,0.0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,15.0,60,1.0,10,1.0,0.0,0.0,1.0,0.0,0.0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,7.0,60,2.0,10,1.0,0.0,0.0,0.0,1.0,0.0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,17.0,60,5.0,3,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972935,24000.0,10.49,515.74,125000.0,10.98,0.0,725.0,729.0,0.0,15.0,...,15.0,60,1.0,4,0.0,1.0,0.0,1.0,0.0,0.0
1972936,24000.0,10.49,515.74,108000.0,34.94,0.0,695.0,699.0,1.0,24.0,...,25.0,60,1.0,10,1.0,0.0,0.0,1.0,0.0,0.0
1972937,40000.0,10.49,859.56,227000.0,12.75,7.0,705.0,709.0,1.0,5.0,...,21.0,60,1.0,9,1.0,0.0,0.0,0.0,0.0,1.0
1972938,24000.0,14.49,564.56,110000.0,18.30,0.0,660.0,664.0,0.0,10.0,...,17.0,60,2.0,6,0.0,0.0,1.0,1.0,0.0,0.0


Similar to before we are going to drop the encoded columns from our categorical data and pick a reference column. In this case, it is going to be "Verified" as it has the least amount of value counts.

In [31]:
# drop Not Verified, Source Verified, Verified columns from categorical_df
categorical_df = categorical_df.drop(['Not Verified', 'Source Verified', 'Verified'], axis=1)

# drop Verified column from numerical_df
numerical_df = numerical_df.drop('Verified', axis=1)

Let's see what columns we have left to process:

In [32]:
categorical_df.head()

,loan_status,purpose,addr_state,initial_list_status,application_type,disbursement_method
0,Paid,debt_consolidation,PA,w,Individual,Cash
1,Paid,small_business,SD,w,Individual,Cash
2,Paid,home_improvement,IL,w,Joint App,Cash
3,Paid,debt_consolidation,NJ,w,Individual,Cash
4,Paid,major_purchase,PA,w,Individual,Cash


### Purpose

Next, we are going to process `purpose`.

*Data Dictionary*
- purpose: A category provided by the borrower for the loan request.

In [33]:
categorical_df['purpose'].value_counts()

debt_consolidation    1125110
credit_card            460263
home_improvement       128022
other                  116007
major_purchase          42291
medical                 22669
car                     19569
small_business          19258
vacation                13047
moving                  12873
house                   11829
renewable_energy         1160
wedding                   840
educational                 2
Name: purpose, dtype: int64

Since there is quite a few values, we are going to combine some of them that are similar. We are going to add 'educational', 'wedding', and 'renewable_energy' to other and combine 'moving', 'house' and 'home_improvement' into 'housing'. This will be done using a mapper.

In [34]:
# mapper
purpose_mapper = {'debt_consolidation': 'debt_consolidation', 
                  'credit_card': 'credit_card', 
                  'home_improvement': 'housing', 
                  'other': 'other', 
                  'major_purchase': 'major_purchase', 
                  'small_business': 'small_business', 
                  'car': 'car', 
                  'medical': 'medical', 
                  'moving': 'housing', 
                  'vacation': 'vacation', 
                  'house': 'housing', 
                  'renewable_energy': 'other', 
                  'wedding': 'other', 
                  'educational': 'other',
                  False: None}

# replace values
categorical_df['purpose'] = categorical_df['purpose'].replace(purpose_mapper)

# get new value counts
categorical_df['purpose'].value_counts()

debt_consolidation    1125110
credit_card            460263
housing                152724
other                  118009
major_purchase          42291
medical                 22669
car                     19569
small_business          19258
vacation                13047
Name: purpose, dtype: int64

Now that we have lowered the number of values in this column, we can use One Hot Encoding to separate them into their own binary columns.

In [35]:
# one hot encode purpose column
purpose_ohe = ohe.fit_transform(categorical_df[['purpose']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(purpose_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)

We can now drop `purpose` from our categorical data and add the encoded columns to the numeric data.

In [36]:
# drop purpose column from categorical_df
categorical_df = categorical_df.drop('purpose', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(purpose_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Now we are going to drop the encoded columns from our categorical data and drop "vacation" from our numeric data as the reference column.

In [37]:
# drop encoded columns from categorical_df
categorical_df = categorical_df.drop(['car', 'credit_card', 'debt_consolidation', 'housing', 'major_purchase', 'medical', 'other', 'small_business', 'vacation'], axis=1)

# drop vacation column from numerical_df
numerical_df = numerical_df.drop('vacation', axis=1)

### State

Next, we can take a look at `state`.

*Data Dictionary*
- state: The state provided by the borrower in the loan application

Let's take a look at the value counts in this feature:

In [38]:
categorical_df['addr_state'].value_counts()

CA    274160
TX    165550
NY    162060
FL    137894
IL     80778
NJ     73343
PA     67191
OH     65577
GA     64539
VA     55024
NC     54082
MI     50017
MD     47311
AZ     46315
MA     45950
CO     42943
WA     41045
MN     35478
IN     33849
CT     31578
TN     31532
MO     31305
NV     27904
WI     26130
SC     23706
AL     23209
OR     23058
LA     22387
KY     18898
OK     18002
KS     16767
AR     14429
UT     13111
MS     10955
NM     10197
NH      9920
HI      9437
RI      8864
WV      7085
NE      7078
DE      5546
MT      5480
DC      4671
AK      4616
ME      4394
VT      4326
WY      4216
SD      4032
ID      3673
ND      3327
IA         1
Name: addr_state, dtype: int64

Although there are a lot of states, there might be a relationship between the state and the number of default loans. Let's compare the state in which the loan too place and the outcome.

In [39]:
# add_state vs loan_status
pd.crosstab(categorical_df['addr_state'], categorical_df['loan_status']).T

addr_state,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
loan_status,,,,,,,,,,,,,,,,,,,,,
Default,553,3297,2070,5468,33364,4060,2990,397,638,17421,...,540,3862,19546,1502,6604,314,3893,2811,635,451
Paid,4063,19912,12359,40847,240796,38883,28588,4274,4908,120473,...,3492,27670,146004,11609,48420,4012,37152,23319,6450,3765


Let's check the correlation between `addr_state` and `loan_status`

In [40]:
# import chi2
from scipy.stats import chi2_contingency

# test between addr_state and loan_status
chi2_contingency(pd.crosstab(categorical_df['addr_state'], categorical_df['loan_status']))

Chi2ContingencyResult(statistic=3047.909488640779, pvalue=0.0, dof=50, expected_freq=array([[5.46866406e+02, 4.06913359e+03],
       [2.74961491e+03, 2.04593851e+04],
       [1.70943141e+03, 1.27195686e+04],
       [5.48702721e+03, 4.08279728e+04],
       [3.24802630e+04, 2.41679737e+05],
       [5.08753988e+03, 3.78554601e+04],
       [3.74110645e+03, 2.78368935e+04],
       [5.53382362e+02, 4.11761764e+03],
       [6.57045297e+02, 4.88895470e+03],
       [1.63365676e+04, 1.21557432e+05],
       [7.64605958e+03, 5.68929404e+04],
       [1.11801956e+03, 8.31898044e+03],
       [1.18471925e-01, 8.81528075e-01],
       [4.35147381e+02, 3.23785262e+03],
       [9.56992517e+03, 7.12080748e+04],
       [4.01015619e+03, 2.98388438e+04],
       [1.98641877e+03, 1.47805812e+04],
       [2.23888244e+03, 1.66591176e+04],
       [2.65223099e+03, 1.97347690e+04],
       [5.44378496e+03, 4.05062150e+04],
       [5.60502525e+03, 4.17059747e+04],
       [5.20565639e+02, 3.87343436e+03],
       [5.925

We can see that that there is no statistical significange between `addr_state` and our target column. Instead of encoding the column and adding 51 new features to our data, we will go ahead and drop it.

In [41]:
# drop addr_state from categorical_df
categorical_df = categorical_df.drop('addr_state', axis=1)

### Sanity Check on Categorical and Numerical

In [42]:
categorical_df.head()

,loan_status,initial_list_status,application_type,disbursement_method
0,Paid,w,Individual,Cash
1,Paid,w,Individual,Cash
2,Paid,w,Joint App,Cash
3,Paid,w,Individual,Cash
4,Paid,w,Individual,Cash


In [43]:
numerical_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,Not Verified,Source Verified,car,credit_card,debt_consolidation,housing,major_purchase,medical,other,small_business
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Initial List Status

Next, we can look at `initial_list_status`.

*Data Dictionary*
- initial_list_status: The initial listing status of the loan. Possible values are – W(whole loans), F(fractional loans)

Let's look at the values in the feature:

In [44]:
categorical_df['initial_list_status'].value_counts()

w    1379903
f     593037
Name: initial_list_status, dtype: int64

We have two values, whole and fractional loans. To make this more clear, we are going to change the names of the values using a mapper.

In [45]:
# mapper
initial_list_status_mapper = {'w': "whole", 'f': "fractional"}

# replace values
categorical_df['initial_list_status'] = categorical_df['initial_list_status'].replace(initial_list_status_mapper)

# get new value counts
categorical_df['initial_list_status'].value_counts()

whole         1379903
fractional     593037
Name: initial_list_status, dtype: int64

Now that the names of the values are clear, we can use One Hot Encoding to create binary columns.

In [46]:
# one hot encode initial_list_status column
initial_list_status_ohe = ohe.fit_transform(categorical_df[['initial_list_status']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(initial_list_status_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)

We can now drop `inital_list_status` from our categorical data and add the encoded columns to the numeric data.

In [47]:
# drop inital_list_status column from categorical_df
categorical_df = categorical_df.drop('initial_list_status', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(initial_list_status_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Now we are going to drop the encoded columns from our categorical data and drop "fractional" from our numeric data as the reference column.

In [48]:
# drop encoded columns from categorical_df
categorical_df = categorical_df.drop(['fractional', 'whole'], axis=1)

# drop fractional column from numerical_df
numerical_df = numerical_df.drop('fractional', axis=1)

### Application Type

Next we have the `application_type`.

*Data Dictionary*
- application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers

In [49]:
categorical_df['application_type'].value_counts()

Individual    1876112
Joint App       96828
Name: application_type, dtype: int64

Since almost all values are Individual appliactions, we could drop the column. But let's check if application type has any impact on loan status.

In [50]:
# application_type vs loan_status
pd.crosstab(categorical_df['application_type'], categorical_df['loan_status']).T

application_type,Individual,Joint App
loan_status,,
Default,228748,4990
Paid,1647364,91838


We can see that Individual applications have a higher percentage of default loans that Joint applications. We are going to One Hot Encode this column.

In [51]:
# one hot encode application_type column
application_type_ohe = ohe.fit_transform(categorical_df[['application_type']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(application_type_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)

We can now drop `application_type` from our categorical data and add the encoded columns to the numeric data.

In [52]:
# drop application_type column from categorical_df
categorical_df = categorical_df.drop('application_type', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(application_type_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Now we are going to drop the encoded columns from our categorical data and drop "Joint App" from our numeric data as the reference column.

In [53]:
# drop encoded columns from categorical_df
categorical_df = categorical_df.drop(['Individual', 'Joint App'], axis=1)

# drop Joint App column from numerical_df
numerical_df = numerical_df.drop('Joint App', axis=1)

### Sanity Check on Categorical and Numerical

In [54]:
categorical_df.head()

,loan_status,disbursement_method
0,Paid,Cash
1,Paid,Cash
2,Paid,Cash
3,Paid,Cash
4,Paid,Cash


In [55]:
numerical_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,car,credit_card,debt_consolidation,housing,major_purchase,medical,other,small_business,whole,Individual
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


### Disbursement Method

Now we can process `disbursement_method`.

*Data Dictionary*
- disbursement_method: The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY

Let's look at the values in this feature:

In [56]:
categorical_df['disbursement_method'].value_counts()

Cash         1903231
DirectPay      69709
Name: disbursement_method, dtype: int64

We have two unique values, Cash and DirectPay. Cash takes up the majority of the data. Let's see how this column compares to `loan_status`.

In [57]:
# disbursement_method vs loan_status   
pd.crosstab(categorical_df['disbursement_method'], categorical_df['loan_status']).T

disbursement_method,Cash,DirectPay
loan_status,,
Default,232547,1191
Paid,1670684,68518


Cash looks to have a much higher percentage of default loans than DirectPay. We are going to One Hot Encode this Column.

In [58]:
# one hot encode disbursement_method column
disbursement_method_ohe = ohe.fit_transform(categorical_df[['disbursement_method']]).toarray()

# combine one hot encoded columns with categorical_df
categorical_df = pd.concat([categorical_df, pd.DataFrame(disbursement_method_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
categorical_df = categorical_df.reset_index(drop=True)

We can now drop `disbursement_method` from our categorical data and add the encoded columns to the numeric data.

In [59]:
# drop disbursement_method column from categorical_df
categorical_df = categorical_df.drop('disbursement_method', axis=1)

# add encoded columns to numerical_df
numerical_df = pd.concat([numerical_df, pd.DataFrame(disbursement_method_ohe, columns=ohe.categories_[0])], axis=1)

# reset index
numerical_df = numerical_df.reset_index(drop=True)

Now we are going to drop the encoded columns from our categorical data and drop "DirectPay" from our numeric data as the reference column.

In [60]:
# drop encoded columns from categorical_df
categorical_df = categorical_df.drop(['Cash', 'DirectPay'], axis=1)

# drop DirectPay column from numerical_df
numerical_df = numerical_df.drop('DirectPay', axis=1)

In [61]:
categorical_df.head()

,loan_status
0,Paid
1,Paid
2,Paid
3,Paid
4,Paid


### Loan Status

Our last column to process is `loan_status` which is our target.

*Data Dictionary*
- loan_status: Current status of the loan

We have already mapped this column to have only Paid and Default loans but let's look at the values.

In [62]:
# value counts
print(categorical_df['loan_status'].value_counts())

Paid       1739202
Default     233738
Name: loan_status, dtype: int64


We can now map Paid loans to "0" and Default loans to "1".

In [63]:
# mapper
loan_status_mapper = {'Paid': 0, 'Default': 1}

# replace values
categorical_df['loan_status'] = categorical_df['loan_status'].replace(loan_status_mapper)

# get new value counts
categorical_df['loan_status'].value_counts()

0    1739202
1     233738
Name: loan_status, dtype: int64

Now that the values are mapped, we are going to convert the column to an integer, drop it from our categorical data and add it to our numerical data.

In [64]:
# change loan_status to int
categorical_df['loan_status'] = categorical_df['loan_status'].astype(int)

In [65]:
# drop loan_status from categorical_df and add to numerical_df
numerical_df['loan_status'] = categorical_df['loan_status']
categorical_df = categorical_df.drop(columns='loan_status')

### Sanity Checks

In [66]:
categorical_df.head()

""
0
1
2
3
4


We have no more columns to process.

In [67]:
numerical_df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,debt_consolidation,housing,major_purchase,medical,other,small_business,whole,Individual,Cash,loan_status
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972935,24000.0,10.49,515.74,125000.0,10.98,0.0,725.0,729.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
1972936,24000.0,10.49,515.74,108000.0,34.94,0.0,695.0,699.0,1.0,24.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
1972937,40000.0,10.49,859.56,227000.0,12.75,7.0,705.0,709.0,1.0,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
1972938,24000.0,14.49,564.56,110000.0,18.30,0.0,660.0,664.0,0.0,10.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1


Rename our fully processed data set:

In [68]:
accepted_processed = numerical_df

### Write to CSV

In [69]:
# save processed data
accepted_processed.to_csv('accepted_processed.csv', index=False)

In [70]:
# read in processed data
accepted_processed = pd.read_csv('accepted_processed.csv')

### Sanity checks

In [71]:
accepted_processed.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,...,debt_consolidation,housing,major_purchase,medical,other,small_business,whole,Individual,Cash,loan_status
0,3600.0,13.99,123.03,55000.0,5.91,0.0,675.0,679.0,1.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
1,24700.0,11.99,820.28,65000.0,16.06,1.0,715.0,719.0,4.0,22.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0
2,20000.0,10.78,432.66,63000.0,10.78,0.0,695.0,699.0,0.0,6.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
3,35000.0,14.85,829.90,110000.0,17.06,0.0,785.0,789.0,0.0,13.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
4,10400.0,22.45,289.91,104433.0,25.37,1.0,695.0,699.0,3.0,12.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0


Everything looks good. Let's check the data types now:

In [72]:
accepted_processed.dtypes.value_counts()

float64    74
int64       3
dtype: int64

We can see that all of our columns are now numeric with majority floats and 3 integers.

Now let's make sure we still don't have any NULL values.

In [73]:
# sum of null values
accepted_processed.isnull().sum().sum()

0

Our data looks good. Now we can move on to removing any highly correlated columns.

## Highly Correlated Columns

The last part of this notebook is to remove any columns with high correlations. To decide on which out of two column we will drop, we will be using the data dictionary. First, we are going to look at all columns that have over 80% correlation.

In [74]:
# create correlation matrix
correlations = accepted_processed.corr()

# set threshold
lower_bound = 0.8
upper_bound = 1.0

# use absolute value and bounds to filter correlations
absolute = correlations.abs().gt(lower_bound) & correlations.abs().lt(upper_bound)

# output the correlations stacked by column
list = correlations.where(absolute).stack()

list.head(10000)

loan_amnt                   installment                   0.944537
int_rate                    grade                         0.954941
installment                 loan_amnt                     0.944537
fico_range_low              fico_range_high               1.000000
fico_range_high             fico_range_low                1.000000
open_acc                    num_op_rev_tl                 0.832667
                            num_sats                      0.998979
revol_bal                   total_rev_hi_lim              0.805710
revol_util                  bc_util                       0.869062
last_fico_range_high        last_fico_range_low           0.845875
last_fico_range_low         last_fico_range_high          0.845875
acc_now_delinq              num_tl_30dpd                  0.801496
tot_cur_bal                 avg_cur_bal                   0.834374
                            tot_hi_cred_lim               0.977303
total_rev_hi_lim            revol_bal                     0.80

Looking at the list, we can drop the columns that we believe will be a worse predictor in our model.

In [75]:
# drop columns 
accepted_processed = accepted_processed.drop(columns=['installment', 'int_rate', 'fico_range_high', 'num_sats', 'bc_util', 
                                                      'collection_recovery_fee', 'tot_hi_cred_lim', 'num_rev_tl_bal_gt_0', 
                                                      'mo_sin_old_rev_tl_op', 'RENT'])
    

Next, we are going to look at all columns that have over 70% correlation.

In [76]:
# create correlation matrix
correlations = accepted_processed.corr()

# set threshold
lower_bound = 0.7
upper_bound = 1.0

# use absolute value and bounds to filter correlations
absolute = correlations.abs().gt(lower_bound) & correlations.abs().lt(upper_bound)

# output the correlations stacked by column
list = correlations.where(absolute).stack()

list.head(10000)

open_acc                    total_acc                     0.720568
                            num_op_rev_tl                 0.832667
pub_rec                     tax_liens                     0.702585
revol_bal                   total_rev_hi_lim              0.805710
revol_util                  percent_bc_gt_75              0.740375
total_acc                   open_acc                      0.720568
                            num_rev_accts                 0.762823
last_fico_range_high        last_fico_range_low           0.845875
last_fico_range_low         last_fico_range_high          0.845875
acc_now_delinq              num_tl_30dpd                  0.801496
tot_cur_bal                 avg_cur_bal                   0.834374
total_rev_hi_lim            revol_bal                     0.805710
                            total_bc_limit                0.753001
acc_open_past_24mths        num_tl_op_past_12m            0.764076
avg_cur_bal                 tot_cur_bal                   0.83

Now we can drop the columns:

In [77]:
accepted_processed = accepted_processed.drop(columns=['open_acc', 'last_fico_range_high', 'num_tl_30dpd', 'avg_cur_bal', 
                                                      'total_rev_hi_lim', 'acc_open_past_24mths', 'bc_open_to_buy', 'num_actv_rev_tl', 
                                                      'num_bc_sats', 'num_rev_accts', 'total_bal_ex_mort'])

Now that we have dropped any columns that have high correlation, let's take a look at the shape of our final data frame:

In [78]:
accepted_processed.shape

(1972940, 56)

Our model ready data has 1.97 million rows and 56 features. Let's save this to a new data frame.

In [79]:
accepted_ready = accepted_processed

## Write to CSV

In [80]:
# to csv
accepted_ready.to_csv('accepted_ready.csv', index=False)

In [81]:
# read csv
accepted_ready = pd.read_csv('accepted_ready.csv')

### Sanity Checks

Since the last time we wrote to a new CSV, the datetime column changed to an object, let's do some sanity checks:

In [82]:
accepted_ready.shape

(1972940, 56)

In [83]:
accepted_ready.head()

,loan_amnt,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,pub_rec,revol_bal,revol_util,total_acc,...,debt_consolidation,housing,major_purchase,medical,other,small_business,whole,Individual,Cash,loan_status
0,3600.0,55000.0,5.91,0.0,675.0,1.0,0.0,2765.0,29.7,13.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
1,24700.0,65000.0,16.06,1.0,715.0,4.0,0.0,21470.0,19.2,38.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0
2,20000.0,63000.0,10.78,0.0,695.0,0.0,0.0,7869.0,56.2,18.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
3,35000.0,110000.0,17.06,0.0,785.0,0.0,0.0,7802.0,11.6,17.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0
4,10400.0,104433.0,25.37,1.0,695.0,3.0,0.0,21929.0,64.5,35.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0


In [84]:
# all null values
accepted_ready.isnull().sum().sum()

0

In [85]:
# data types
accepted_ready.dtypes.value_counts()

float64    53
int64       3
dtype: int64

Our data is now fully processed and we can move onto modeling. This can be seen in our third notebook.